# Gene ontologies

lamindb provides access to the following public gene ontologies:

1. [Ensembl](https://ensembl.org)
2. [NCBI Gene](https://www.ncbi.nlm.nih.gov/gene)

## Setup

In [ ]:
!lamin init --storage ./test-ontologies --schema bionty

In [ ]:
import lnschema_bionty as lb
import pandas as pd

# adds an entry "human" into an empty instance
lb.settings.organism = "human"

## Bionty objects

Let us create a public knowledge accessor with {meth}`~lnschema_bionty.dev.BioRegistry.bionty`, which chooses a default public knowledge source from {meth}`~lnschema_bionty.BiontySource`. It's a [Bionty](https://lamin.ai/docs/bionty/bionty.bionty) object, which you can think about as a less-capable registry:

In [ ]:
gene_bt = lb.Gene.bionty(organism="human")
gene_bt

As for registries, you can export the ontology as a `DataFrame`:

In [ ]:
df = gene_bt.df()
df.head()

Unlike registries, you can also export it as a Pronto object via `gene_bt.ontology`.

## Look up terms

As for registries, terms can be looked up with auto-complete:

In [ ]:
lookup = gene_bt.lookup()

The `.` accessor provides normalized terms (lower case, only contains alphanumeric characters and underscores):

In [ ]:
lookup.tcf7

To look up the exact original strings, convert the lookup object to dict and use the `[]` accessor:

In [ ]:
lookup_dict = lookup.dict()
lookup_dict["TCF7"]

By default, the `name` field is used to generate lookup keys. You can specify another field to look up:

In [ ]:
lookup = gene_bt.lookup(gene_bt.ncbi_gene_id)

If multiple entries are matched, they are returned as a list:

In [ ]:
lookup.bt_100126572

## Search terms

Search behaves in the same way as it does for registries:

In [ ]:
gene_bt = lb.Gene.bionty()
gene_bt.search("TP53").head(3)

By default, search also covers synonyms:

In [ ]:
gene_bt.search("PDL1").head(3)

You can turn this off synonym by passing `synonyms_field=None`:

In [ ]:
gene_bt.search("PDL1", synonyms_field=None).head(3)

Search another field (default is `.name`):

In [ ]:
gene_bt.search(
    "tumor protein p53", field=gene_bt.description
).head()

## Standardize gene identifiers

Let us generate a `DataFrame` that stores a number of gene identifiers, some of which corrupted:

In [ ]:
data = {
    "gene symbol": ["A1CF", "A1BG", "FANCD1", "corrupted"],
    "ncbi id": ["29974", "1", "5133", "corrupted"],
    "ensembl_gene_id": [
        "ENSG00000148584",
        "ENSG00000121410",
        "ENSG00000188389",
        "ENSGcorrupted",
    ],
}
df_orig = pd.DataFrame(data).set_index("ensembl_gene_id")
df_orig

First we can check whether any of our values are validated against the ontology reference:

In [ ]:
validated = gene_bt.validate(df_orig.index, gene_bt.ensembl_gene_id)
df_orig.index[~validated]

Next, we validate which symbols are mappable against the ontology:

In [ ]:
# based on NCBI gene ID
gene_bt.validate(df_orig["ncbi id"], gene_bt.ncbi_gene_id)

In [ ]:
# based on Gene symbols
validated_symbols = gene_bt.validate(df_orig["gene symbol"], gene_bt.symbol)
df_orig["gene symbol"][~validated_symbols]

Here, 2 of the gene symbols are not validated. Inspect why:

In [ ]:
gene_bt.inspect(df_orig["gene symbol"], gene_bt.symbol);

Logging suggests to use `.standardize()`:

In [ ]:
mapped_symbol_synonyms = gene_bt.standardize(df_orig["gene symbol"])
mapped_symbol_synonyms

Optionally, you can return a mapper in the form of `{synonym1: standardized_name1, ...}`:

In [ ]:
gene_bt.standardize(df_orig["gene symbol"], return_mapper=True)

We can use the standardized symbols as the new standardized index:

In [ ]:
df_curated = df_orig.reset_index()
df_curated.index = mapped_symbol_synonyms
df_curated

You can convert identifiers by passing `return_field` to {meth}`~lamindb.dev.CanValidate.standardize`:

In [ ]:
gene_bt.standardize(
    df_curated.index, field=gene_bt.symbol, return_field=gene_bt.ensembl_gene_id,
)

And return mappable identifiers as a dict:

In [ ]:
gene_bt.standardize(
    df_curated.index,
    field=gene_bt.symbol,
    return_field=gene_bt.ensembl_gene_id,
    return_mapper=True,
)

## Ontology source versions

For any given entity, we can choose from a number of versions:

In [ ]:
lb.BiontySource.filter(entity="Gene").df()

When instantiating a Bionty object, we can choose a source or version:

In [ ]:
bionty_source = lb.BiontySource.filter(source="ensembl", version="release-110", organism="human").one()
gene_bt = lb.Gene.bionty(bionty_source=bionty_source)
gene_bt

The currently used ontologies can be displayed using:

In [ ]:
lb.BiontySource.filter(currently_used=True).df()